## 1. Introduction: 
This Jupyter Notebook performs a proportions test using `proportions_ztest` from `statsmodels.stats.proportion` to determine if the amount paid for freight affects whether or not the shipment was late.

### 1.1 Problem Statement
**$H_0$**: The proportion of expensive shipments is the same for reasonable shipment for late shipments.

**$H_a$**: The proportion of expensive shipments is more than reasonable shipment for late shipments.

### 1.2 Data Source
The dataset, `late_shipments.feather` is from datacamp. This is available as a feather file for datacamp subscribers and is expected to be placed in `../../data` folder.  
We will be using columns: 

    - `late` column which is a `Yes, No` column. Yes representing that the shipment was late,  
    - `freight_cost_groups` Freight costs are stored as categories `expensive` and `reasonable`

## 2. Import Libraries and Set Significe Level
This step imports the necessary Python libraries and loads the dataset into a pandas DataFrame, and sets the significance level to 0.05
- `pandas` is imported as pd for data manipulation and analysis.
- `proportions_ztest` is imported to test for proportions based on normal (z) test
- `alpha` is set to 0.05. This is the chosen significance level, which will be used to compare against the p-value to decide whether to reject the null hypothesis.

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
alpha = 0.05

### 2.1 Load Data and preprocessing
- Load the `late_shipments.feather` dataset into a pandas DataFrame as `late_shipments`.
- Inspect `late` and `freight_cost_groups` column values using `value_counts` method.

In [8]:
late_shipments = pd.read_feather('../../data/late_shipments.feather')

late_shipments['late'].value_counts()

late
No     939
Yes     61
Name: count, dtype: int64

In [9]:
late_shipments['freight_cost_groups'].value_counts()

freight_cost_groups
expensive     531
reasonable    455
Name: count, dtype: int64

## 3. Calculating Sample Proportions

To perform the z-test for proportions, we need the number of "successes" (late shipments) and the total number of observations (total number of shipments) for each group.

1.  **Count the successes and total observations for each group.** A "success" is defined as a shipment beign late. We'll count the number of shipments where the `late` column is `Yes`.

2.  **Calculate the number of users for each group.** The `sum` of each group gives number of shipments in that group.

In [10]:
# Count the late column values for each freight_cost_group
late_by_freight_cost_group = late_shipments.groupby('freight_cost_groups')['late'].value_counts()
print(late_by_freight_cost_group)
# Create an array of the "Yes" counts for each freight_cost_group
count_success = np.array([late_by_freight_cost_group[('expensive', 'Yes')], late_by_freight_cost_group[('reasonable', 'Yes')]])
print(count_success)
# Create an array of the total number of rows in each freight_cost_group
n_obs = np.array([late_by_freight_cost_group['expensive'].sum(), late_by_freight_cost_group['reasonable'].sum()])
print(n_obs)

freight_cost_groups  late
expensive            No      489
                     Yes      42
reasonable           No      439
                     Yes      16
Name: count, dtype: int64
[42 16]
[531 455]



## 4. Performing the Z-Test

The `proportions_ztest` function from `statsmodels.stats.proportion` is the primary tool for this test. It takes three key arguments:

  * `count`: A list or array of the number of successes for each group.
  * `nobs`: A list or array of the total number of observations for each group.
  * `alternative`: Specifies the type of test. For our hypothesis, the `alternative='larger'`.

The function returns two values: the **z-statistic** and the **p-value**.


In [11]:

# Perform the z-test
z_score, p_value = proportions_ztest(count=count_success, nobs=n_obs, alternative='larger')

# Print the results
print(f"z-score: {z_score:.4f}")
print(f"p-value: {p_value:.4f}")

z-score: 2.9226
p-value: 0.0017




### Interpreting the Results

The p-value is the probability of observing our data (or more extreme data) if the null hypothesis were true. We compare the p-value to our **significance level** ($\\alpha$), typically 0.05.

  * If **p-value \< $\alpha$**: We **reject the null hypothesis** ($H_0$). This suggests that proportion of expensive shipments is more than reasonable shipment for late shipments.
  * If **p-value \>= $\alpha$**: We **fail to reject the null hypothesis**. This suggests there is not enough evidence to conclude that proportion of expensive shipments is more than reasonable shipment for late shipments.


In [12]:
# Interpret the results
if p_value < alpha:
    print("Result: Reject the null hypothesis. The proportion of late expensive shipments is more than  late reasonable shipments.")
else:
    print("Result: Fail to reject the null hypothesis. Sufficient evidence not found to reject null hypothesis.")

Result: Reject the null hypothesis. The proportion of late expensive shipments is more than  late reasonable shipments.
